# **Scraper des tables de matières (Gallica)**

Ce scraper permet de récupérer automatiquement la table des matières d'un document sur Gallica.

*Document d'entrée* : fichier excel (.xlsx) contenant les liens vers les documents dans Gallica, par exemple : https://gallica.bnf.fr/ark:/12148/bpt6k5529888n.

*Document de sortie* : fichier texte (.txt) pour chaque lien de document Gallica dans lequel se trouve le sommaire.

In [ ]:
#@markdown # Connexion du notebook à son compte Google Drive et signalement du dossier de travail :

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''

## Installation des bibliothèques et connexion au compte Google Drive

from google.colab import drive
from openpyxl import load_workbook
import pandas as pd
import re
import requests
import shutil
import os
import xml.etree.ElementTree as ET
import lxml
from lxml import etree
from re import match
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import string
from urllib.error import HTTPError

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

#@markdown ###Indiquer le chemin vers le dossier de travail sur le Google Drive :
chemin_vers_dossier_travail = '/content/drive/My Drive/gallica'#@param {type:"string"}

if not os.path.exists(chemin_vers_dossier_travail):
      os.makedirs(chemin_vers_dossier_travail)
os.chdir(chemin_vers_dossier_travail)

In [2]:
#@markdown # Récupération des tables des matières des documents Gallica : 

#@markdown ###Indiquer le chemin vers le fichier Excel (.xlsx) contenant les liens Gallica :
chemin_vers_fichier_excel = '/content/drive/My Drive/gallica/documents.xlsx' #@param {type:"string"}

#@markdown ###Indiquer le nom de la feuille du fichier Excel (il s'agit des onglets en bas du document) :
nom_feuille = 'Feuille 1' #@param {type:"string"}

#@markdown ###Indiquer la lettre de la colonne avec le lien ou identifiant ARK sur Gallica :
lettre_colonne = 'A' #@param {type:"string"}

def table_mat(fichier_excel):

  """ Fonction qui télécharge sous forme de fichier .txt les tables des matières 
  des documents dont l'identifiant est indiqué dans un fichiers .xlsx. 
  La fonction utilise l'API Gallica dédiée """

  wb= load_workbook(chemin_vers_fichier_excel)
  ws= wb[nom_feuille]
  column_id = ws[lettre_colonne]
  id_list = [column_id[x].value for x in range(len(column_id))]
  for e in id_list:
    if e == None:
      exit()
    else: 
      match = re.findall(r"b[a-z0-9]{4,}", e)
      for id in match:
        url = "".join(["https://gallica.bnf.fr/services/Toc?ark=ark:/12148/",id])
        request = urllib.request.Request(url)
        try:
          content = urllib.request.urlopen(request)
          parse = BeautifulSoup(content, 'html.parser')
          text1 = parse.find_all('seg')
          filename = "".join([id,"_table_des_matieres.txt"])
          text2 = re.sub('<.*?>', '', str(text1))
          text5 = re.sub("\d+", "", text2)
          with open(filename, 'w') as text_file:
            text_file.write(str(text5))
        except HTTPError as tp:
          print("Le contenu de ce lien n'a pas pu être téléchargé : " + id)

table_mat(chemin_vers_fichier_excel)